## Analyze whether SNWD varies more from year to year or from place to place.

In [1]:
import pandas as pd
import numpy as np
import urllib
import math

In [2]:
import findspark
findspark.init()

from pyspark import SparkContext
#sc.stop()
sc = SparkContext(master="local[3]",pyFiles=['lib/numpy_pack.py','lib/spark_PCA.py','lib/computeStats.py'])

from pyspark import SparkContext
from pyspark.sql import *
sqlContext = SQLContext(sc)

In [3]:
import sys
sys.path.append('./lib')

import numpy as np
from numpy_pack import packArray,unpackArray
from spark_PCA import computeCov
from computeStats import computeOverAllDist, STAT_Descriptions

In [4]:
### Read the data frame from pickle file

data_dir='../../Data/Weather'
file_index='SSSSSSBB'
meas='SNWD'

from pickle import load

#read statistics
filename=data_dir+'/STAT_%s.pickle'%file_index
STAT,STAT_Descriptions = load(open(filename,'rb'))
print('keys from STAT=',STAT.keys())

#!ls -ld $data_dir/*.parquet

#read data
filename=data_dir+'/decon_%s_%s.parquet'%(file_index,meas)

df=sqlContext.read.parquet(filename)
print(df.count())
df.show(2)

('keys from STAT=', ['TMIN', 'TOBS', 'TMAX', 'SNOW', 'SNWD', 'PRCP'])
23
+------------------+------------------+--------------------+---------+--------+--------+---------+-----------+--------------------+--------------------+--------------------+------------------+-----------+---------+------+--------------------+------+
|           coeff_1|           coeff_2|             coeff_3|elevation|   label|latitude|longitude|measurement|               res_1|               res_2|               res_3|          res_mean|    station|total_var|undefs|              vector|  year|
+------------------+------------------+--------------------+---------+--------+--------+---------+-----------+--------------------+--------------------+--------------------+------------------+-----------+---------+------+--------------------+------+
|0.6769542357723421|470.58763560573317|-0.16808421295183326|    780.9|SSSSSSBB| 34.0333|-116.5833|       SNWD|  0.9999979331361124|0.001202638947196...|0.001202511581622...|0.99

In [5]:
print df.columns

['coeff_1', 'coeff_2', 'coeff_3', 'elevation', 'label', 'latitude', 'longitude', 'measurement', 'res_1', 'res_2', 'res_3', 'res_mean', 'station', 'total_var', 'undefs', 'vector', 'year']


In [6]:
#extract longitude and latitude for each station
feature='coeff_1'
sqlContext.registerDataFrameAsTable(df,'weather')
#Features=', '.join(['coeff_1', 'coeff_2', 'coeff_3', 'elevation', 'latitude', 'longitude',\
#          'res_1', 'res_2', 'res_3', 'res_mean', 'year'])
Features='station, year, coeff_2'
Query="SELECT %s FROM weather"%Features
print(Query)
pdf = sqlContext.sql(Query).toPandas()
pdf.head()

SELECT station, year, coeff_2 FROM weather


station    year     coeff_2
0  USC00045863  1949.0  470.587636
1  USC00046196  1967.0    0.202109
2  USC00045502  2007.0   28.499771
3  USC00045863  1962.0    6.989088
4  USC00029287  1916.0   -0.257934

In [7]:
year_station_table=pdf.pivot(index='year', columns='station', values='coeff_2')
year_station_table.head(10)

station  USC00020060  USC00021282  USC00021365  USC00027453  USC00028489  \
year                                                                       
1916.0           NaN          NaN          NaN          NaN          NaN   
1917.0           NaN          NaN          NaN          NaN          NaN   
1937.0     -0.261707          NaN          NaN          NaN    14.193238   
1949.0           NaN          NaN          NaN          NaN          NaN   
1951.0           NaN          NaN          NaN          NaN          NaN   
1955.0           NaN          NaN          NaN          NaN          NaN   
1957.0           NaN          NaN          NaN          NaN          NaN   
1960.0           NaN          NaN          NaN          NaN          NaN   
1962.0           NaN          NaN    11.827737          NaN          NaN   
1964.0     -0.255649          NaN          NaN    -0.257914          NaN   

station  USC00029287  USC00042327  USC00043855  USC00045502  USC00045863  \
year                                                                       
1916.0     -0.257934          NaN          NaN          NaN          NaN   
1917.0     -0.255657          NaN          NaN          NaN          NaN   
1937.0           NaN          NaN          NaN          NaN          NaN   
1949.0           NaN          NaN    14.839124          NaN   470.587636   
1951.0           NaN          NaN          NaN          NaN    -0.255649   
1955.0           NaN          NaN          NaN          NaN    21.891062   
1957.0           NaN          NaN          NaN          NaN    -0.396749   
1960.0           NaN          NaN          NaN          NaN    -0.305861   
1962.0           NaN          NaN          NaN          NaN     6.989088   
1964.0           NaN          NaN          NaN          NaN          NaN   

station  USC00046196  
year                  
1916.0           NaN  
1917.0           NaN  
1937.0           NaN  
1949.0           NaN  
1951.0           NaN  
1955.0           NaN  
1957.0           NaN  
1960.0           NaN  
1962.0           NaN  
1964.0     -0.445516

### Estimating the effect of the year vs the effect of the station

To estimate the effect of time vs. location on the first eigenvector coefficient we
compute:

* The average row: `mean-by-station`
* The average column: `mean-by-year`

We then compute the RMS before and after subtracting either  the row or the column vector.

In [8]:
def RMS(Mat):
    return np.sqrt(np.nanmean(Mat**2))

mean_by_year=np.nanmean(year_station_table,axis=1)
mean_by_station=np.nanmean(year_station_table,axis=0)
tbl_minus_year = (year_station_table.transpose()-mean_by_year).transpose()
tbl_minus_station = year_station_table-mean_by_station

print 'total RMS                   = ',RMS(year_station_table)
print 'RMS removing mean-by-station= ',RMS(tbl_minus_station)
print 'RMS removing mean-by-year   = ',RMS(tbl_minus_year)

total RMS                   =  98.5450290367
RMS removing mean-by-station=  90.0794848834
RMS removing mean-by-year   =  67.2340744418


In [9]:
T=year_station_table
print 'initial RMS=',RMS(T)
for i in range(5):
    mean_by_year=np.nanmean(T,axis=1)
    T=(T.transpose()-mean_by_year).transpose()
    print i,'after removing mean by year    =',RMS(T)
    mean_by_station=np.nanmean(T,axis=0)
    T=T-mean_by_station
    print i,'after removing mean by stations=',RMS(T)

initial RMS= 98.5450290367
0 after removing mean by year    = 67.2340744418
0 after removing mean by stations= 55.4172871814
1 after removing mean by year    = 49.8503643656
1 after removing mean by stations= 46.4079680604
2 after removing mean by year    = 43.8317005104
2 after removing mean by stations= 41.7049391043
3 after removing mean by year    = 39.8683324655
3 after removing mean by stations= 38.2441704886
4 after removing mean by year    = 36.784656273
4 after removing mean by stations= 35.4560420116
